<a href="https://colab.research.google.com/github/engenheirorodrigoviannini/optical_character_recognition/blob/main/optical_character_recognition_20230620.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


**PRODUCTION ENGINEER | DATA SCIENTIST**


Rodrigo Viannini


Date: 2023-06-20


https://www.linkedin.com/in/rodrigo-viannini-datascientist/

**SETTINGS**

In [1]:
# Install Azure
!pip install --upgrade azure-cognitiveservices-vision-computervision;

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.5/174.5 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 5.2 MB/s eta 0:00:00


In [10]:
# Install Google
!pip install google-cloud-vision;

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 444.0/444.0 kB 9.7 MB/s eta 0:00:00


In [15]:
# Install Paddle Paddle
!python3 -m pip install paddlepaddle;
!pip install "paddleocr>=2.0.1";

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.7/121.7 MB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 64.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.2/383.2 kB 36.9 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-api-core 2.11.0 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.19.5, but you have protobuf 3.20.0 which is incompatible.
google-cloud-bigquery 3.9.0 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.19.5, b

In [16]:
# Imports
import os
import sys
import time
import pandas as pd
import re

# Azure
from azure.cognitiveservices.vision.computervision import ComputerVisionClient
from azure.cognitiveservices.vision.computervision.models import OperationStatusCodes
from azure.cognitiveservices.vision.computervision.models import VisualFeatureTypes
from msrest.authentication import CognitiveServicesCredentials

# Google
from google.cloud import vision

# Paddle
import urllib.request
from PIL import Image
import cv2
import shutil
from paddleocr import PaddleOCR,draw_ocr

In [17]:
# Cleanup and exception list
def clean_ocr_text(ocr):
    """ This function performs text cleaning using regular expressions (RegEX) and replace function.
     This function performs text cleaning using regular expressions (RegEX) and replace function. """

    # Exception list
    del_ = ['Cabo', 'Caber', 'Cab', 'Cont', 'Est',
            'ey', 'Com', 'Proc', 'Cool', 'Ext',
            'Co', 'abo', 'OPR', 'Rst', 'Fst', 'nt',
            'Cahn', 'oi','Caha', 'Caho',
            'Cant', 'ANATE', 'ANAT', 'Et', 'Came',
            'CABO ÓPTICO', 'V TAL', 'VTAL', '\n',
            'Caha:', 'Caha: ', 'queLa']

    text = str(ocr)
    for item in del_:
        # Character replacement
        text = text.replace(str(item) , '').strip()
        text = text.replace(' ','')
        text = text.replace("&","8")
        text = text.replace("|","I")
        text = text.replace("×","X")
        text = text.replace("g","9")
        text = re.sub(r"[][().:;_\"'ـه₦-]","", text)

    return text

**MICROSOFT AZURE**

In [3]:
# Credentials
subscription_key = 'INSERT AZURE KEY'
endpoint = 'https://visionleitura.cognitiveservices.azure.com/'
computervision_client = ComputerVisionClient(endpoint, CognitiveServicesCredentials(subscription_key))

In [9]:
# Azure
def extract_azure_ocr(filepath):
    """ This function returns OCR (Optical character recognition) which is the process that
    converts text in image format to text in human-readable character format. """

    # Open the file in binary format for reading
    content = open(filepath, 'rb')
    try:
        # Call the Azure Computer Vision client to read the file content
        # Set the raw parameter to True to return the raw response
        read_response = computervision_client.read_in_stream(content, raw=True)

        # Get the operation location from the response headers
        read_operation_location = read_response.headers["Operation-Location"]

        # Extract the operation ID from the operation location
        operation_id = read_operation_location.split("/")[-1]

        # Loop to check the status of the read operation
        while True:
            # Get the read result using the operation ID
            read_result = computervision_client.get_read_result(operation_id)

            # Check if the status is not 'notstarted' or 'running' to exit the loop
            if read_result.status.lower() not in ['notstarted', 'running']:
                break

            # Wait for a short interval before checking the status again
            time.sleep(0.2)

        if read_result.status == OperationStatusCodes.succeeded:
            # Initialize an empty string to store the extracted text
            texts = ''

            # Iterate over the read results and extract the text from each line found
            for text_result in read_result.analyze_result.read_results:
                for line in text_result.lines:
                    texts = texts + ' ' + line.text

            # Call the clean_ocr_text function to clean the extracted text
            return clean_ocr_text(texts)
        else:
            # If the operation status is not successful, print a message indicating the unexpected status
            print('Azure: unexpected status (', read_result.status, ') for file:', filepath)
    except:
        # If an exception occurs during the OCR request process, print a failure message
        print('Azure: fail to request OCR for file:', filepath)

    # Return an empty string if no OCR is extracted or an issue occurs
    return ''


In [8]:
image = 'add your image path'
displaying_text = extract_azure_ocr(image)
print(displaying_text)

**GOOGLE**

In [13]:
# Credentials
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "INSERT JSON FILE PATH PROVIDED BY GOOGLE - OCR KEY"
client = vision.ImageAnnotatorClient()

In [14]:
def extract_google_ocr(filepath):
    """This function returns OCR (Optical Character Recognition) which is the process that
    converts text in image format to text in human-readable character format."""

    # Read the image file in binary mode
    with open(filepath, 'rb') as image_file:
        content = image_file.read()

    try:
        # Create a vision.Image object with the image content
        image = vision.Image(content=content)

        # Perform text detection using the Google Vision client
        response = client.text_detection(image=image)
        annotations = response.text_annotations

        texts = []
        for text in annotations:
            texts.append(text.description)

        # Clean and return the first extracted text
        return clean_ocr_text(texts[0])

    except:
        # If an exception occurs during the OCR request process, print a failure message
        print('Google: fail to request OCR for:', filepath)

    # Return an empty string if no OCR is extracted or an issue occurs
    return ''


In [ ]:
image = 'add your image path'
displaying_text = extract_google_ocr(image)
print(displaying_text)

**PADDLE**

In [18]:
def extract_paddle_ocr(filepath):
    """This function returns OCR (Optical Character Recognition) which is the process that
    converts text in image format to text in human-readable character format."""

    # Initialize variables
    img = filepath
    plaqueta_paddle = []
    texts = ''

    # Read the image using OpenCV
    img = cv2.imread(img, cv2.IMREAD_UNCHANGED)

    # Convert the image to grayscale
    filepath_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Apply adaptive thresholding to enhance text visibility
    gauss = cv2.adaptiveThreshold(filepath_gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 33, 9)

    # Initialize PaddleOCR with angle classification and English language
    ocr = PaddleOCR(use_angle_cls=True, lang='en') # need to run only once to download and load model into memory

    try:
        # Perform OCR on the preprocessed image
        result = ocr.ocr(gauss, cls=True)

        # Extract the recognized text from the OCR result
        for idx, line in enumerate(result):
            for x in line:
                texts = texts + x[-1][0]

        plaqueta_paddle.append(texts)
        texts = clean_ocr_text(plaqueta_paddle[0])

        if texts != '':
            msg_result = 'OK'
        else:
            msg_result = 'VAZIO'

    except:
        # If an exception occurs during the OCR request process, print a failure message
        print('Paddle: fail to request OCR for:', filepath)
        msg_result = 'request_erro'
        texts = ''

    # Return the extracted text and a message indicating the result status
    return texts, msg_result


In [ ]:
image = 'add your image path'
displaying_text = extract_paddle_ocr(image)
print(displaying_text)